In [0]:
%run ../_utils

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql.functions import col, count


# Camada GOLD

Na camada gold, as limpezas e ajustes já foram feitos, então essa camada é responsável por aplicar regras de negócio, agregações e junções de dados que convirjam para analises.

Essa é uma tabela sumarizada analítica.

O objetivo dela é informar os meses em que tiveram mais pedidos

In [0]:
tb_name = "olist_gold.total_orders_month_year"
dataset_location = "olist_total_orders_dataset"
target_location = f"dbfs:/FileStore/delta/brazilian_ecommerce/{dataset_location}/gold"

## 1 - Data ingestion


In [0]:
df_orders = spark.read.table("olist_gold.orders") # leituira da delta table central, orders (ja da propria gold)


## 2 - preparation

In [0]:
df = df_orders.groupBy("order_purchase_year", "order_purchase_month").agg(
    count("order_id").alias("total_orders_by_month_year"),
).orderBy("order_purchase_year", "order_purchase_month")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-422770193078593>, line 1
----> 1 df = df_orders.groupBy("year", "month").agg(
      2     count("order_id").alias("total_orders_by_month_year"),
      3 ).orderBy("year", "month")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/group.py:188, in GroupedData.agg(self, *exprs)
    186     assert all(isinstance(c, Column) for c in exprs), "all exprs should be Column"
    187     exprs = cast(Tuple[Column, ...], exprs)
--> 188     jdf = self._jgd.agg(exprs[0]._jc, _t

In [0]:
display(df)

month_year,total_orders_by_month_year
02-2017,1788
01-2018,7313
05-2018,6882
07-2017,4068
08-2018,6514
04-2017,2411
05-2017,3737
08-2017,4374
09-2018,16
02-2018,6814



## Saving data

In [0]:
save_dataframe(df, format_mode="delta", table_name=tb_name, target_location=target_location)

[LOG] Saving olist_gold.total_orders_month_year delta on dbfs:/FileStore/delta/brazilian_ecommerce/olist_total_orders_dataset/gold... OK!



## create delta table

TODO: implementar UPSERT

o upsert serve para não precisar reescrever todos os dados, mas aproveitar do Delta para fazer um MERGE, caso um registro antigo tenha uma nova versão e INSERT para os dados que são novos

In [0]:
create_table(table_name=tb_name, target_location=target_location)

[LOG] Creating delta table olist_gold.total_orders_month_year on dbfs:/FileStore/delta/brazilian_ecommerce/olist_total_orders_dataset/gold... OK!


In [0]:
# exit para fechar a execução
dbutils.notebook.exit("OK")

In [0]:
%sql

select * from olist_gold.orders